In [6]:
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import math
from transformers import BertTokenizer, GPT2LMHeadModel
import numpy as np
from tqdm import trange

In [3]:
model_path = "model\\best_model\\"
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
CANARY = '我的单号是541684'
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('using device:{}'.format(device))
model.eval()
model.to(device)

using device:cpu


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [8]:

def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float("Inf")):
    """Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Args:
        logits: logits distribution shape (vocabulary size)
        top_k > 0: keep only top k tokens with highest probability (top-k filtering).
        top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
            Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert (
        logits.dim() == 1
    )  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

def sample_sequence_batch(
    model,
    context,
    length,
    n_ctx,
    tokenizer,
    temperature=1.0,
    top_k=30,
    top_p=0.0,
    repetition_penalty=1.0,
    device="cpu",
):
    context = torch.tensor(context, dtype=torch.long, device=model.device)
    generated = context
    # print('sigma = ', sigma)

    with torch.no_grad():
        for _ in trange(length):
            # inputs = {"input_ids": generated[:, -(n_ctx - 1) :], "sigma": sigma}
            # outputs = model(**inputs)
            outputs = model(input_ids=generated[:, -(n_ctx - 1) :])
            next_token_logits = outputs[0][:, -1, :]

            for idx, gen in enumerate(generated):
                for id in set(gen):
                    next_token_logits[idx, id] /= repetition_penalty

            next_token_logits = next_token_logits / temperature
            next_token_logits[:, tokenizer.convert_tokens_to_ids("[UNK]")] = -float("Inf")
            filtered_logits = torch.stack([top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p) for logits in next_token_logits])

            next_tokens = torch.multinomial(torch.nn.functional.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_tokens), dim=1)
    
    return generated.tolist()

def is_word(word):
    for item in list(word):
        if item not in "qwertyuiopasdfghjklzxcvbnm":
            return False
    return True

def calculate_ppl(inputs, model):
    device = model.device
    input_ids = inputs.to(device)
    label_ids = input_ids.clone()

    with torch.no_grad():
        output = model(input_ids=input_ids, labels=label_ids)
        loss = output.loss
        sequence_lengths = len(input_ids)
        ppl_steps = torch.exp(loss / sequence_lengths)
        # ppl_steps = torch.exp(loss)

    return ppl_steps.cpu().numpy()

In [9]:

raw_text = "我的手机号是15656"
# 10, 5
nsamples = 5
batch_size = 1
length = 30
temperature = 0.5
repitition_penalty = 5
top_k = 20
top_p = 0
n_ctx = 1024

eps = 0.5
C = 1.0
delta = 1e-5
sigma = 0.810546875
sigma = 0.0
print('sigma = ', sigma)

save_samples = True

if save_samples:
    # if not os.path.exists(save_samples_path):
    #     os.makedirs(save_samples_path)
    samples_file = open("samples.txt", "w", encoding="utf8")

while True:
    
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text))
    generated = 0
    for _ in range(nsamples // batch_size):
        # seed_everything(int(time.time() * 1000) % (2**32 - 1))
        context_tokens_batch = [context_tokens for _ in range(batch_size)]
        out = sample_sequence_batch(
            model,
            context_tokens_batch,
            length,
            n_ctx,
            tokenizer=tokenizer,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repitition_penalty,
            device=device,
        )
        
        for i in range(batch_size):
            generated += 1
            
            text = tokenizer.convert_ids_to_tokens(torch.tensor(out[i]))
            ppl = calculate_ppl(torch.tensor(out[i], dtype=torch.long, device=model.device), model)
            print('ppl = ', ppl)
            
            for it, item in enumerate(text[:-1]):  # 确保英文前后有空格
                if is_word(item) and is_word(text[it + 1]):
                    text[it] = item + " "
            for it, item in enumerate(text):
                if item == "[MASK]":
                    text[it] = ""
                elif item == "\n":
                    text[it] = ""
                elif item == "[CLS]":
                    text[it] = "\n\n"
                elif item == "[SEP]":
                    text[it] = "\n"
            info = "=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40 + "\n"
            print(info)
            text = "".join(text).replace("##", "").strip()
            print(text)
            if save_samples:
                samples_file.write(info)
                samples_file.write(text)
                samples_file.write("\n")
                samples_file.write("=" * 90)
                samples_file.write("\n" * 2)
    print("=" * 80)
    
    break
samples_file.close()

sigma =  0.0


100%|██████████| 30/30 [00:02<00:00, 13.29it/s]


ppl =  1.0720904
======================================== SAMPLE 1 ========================================

我的手机号是15656218990，不能通过微信联系。但在网上查询后发现这个qq账户还有另


100%|██████████| 30/30 [00:02<00:00, 12.59it/s]


ppl =  1.0725421
======================================== SAMPLE 2 ========================================

我的手机号是1565668888，对方会在当天下午打给你。
如果真有人拿着这张卡到处跑了


...

ppl =  1.0785694
======================================== SAMPLE 3 ========================================

我的手机号是15656485142，电话也不多。当时他们在一起谈了好几个月吧！最后还没有


100%|██████████| 30/30 [00:02<00:00, 12.44it/s]


ppl =  1.0701098
======================================== SAMPLE 4 ========================================

我的手机号是15656219766，如果你不想让他知道自己被骗了。
.3、这个女孩子在网


100%|██████████| 30/30 [00:02<00:00, 13.15it/s]


ppl =  1.060725
======================================== SAMPLE 5 ========================================

我的手机号是15656788233，可以在线咨询。
2、拨打电话或者到银行柜台办理信用卡
